# Neural Network hyperparameter tuning 

In this section of the project the NN will be defined with their optimal hyperparameters using Keras Tuner (https://keras.io/keras_tuner/).
There will be one neural network for each subset of features (DataAP, dataPHY and dataALL)

## Loading the libraries

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import pickle as pk
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

## Loading the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing the dataframes dataAP and dataPHY:

- dataAP: data containing the original variables that best correlate with the label data of the dataframe.

- dataPHY: data containing variables from the blood test that are of interest.

- dataALL: Both dataAP and dataPHY combined

In [3]:
#Importing dataAP
file_name = '/content/drive/MyDrive/TFG/dataAP.csv'
dataAP = pd.read_csv(file_name, index_col=[0])

In [4]:
#Importing dataPHY
file_name = '/content/drive/MyDrive/TFG/dataPHY.csv'
dataPHY = pd.read_csv(file_name, index_col=[0])

In [5]:
#Importing dataALL
file_name = '/content/drive/MyDrive/TFG/dataALL.csv'
dataALL = pd.read_csv(file_name, index_col=[0])

One dataframe (**dataAP_g**) will contain the label data in continuous form (Masa_VAT_g) and the other (**dataAP_cat**) will contain the label in
categorical form (Masa_VAT_cat), as stated in the "*Transorming the label data into categotical data*" section from the preprocessing. This is done to avoid confusion when selecting the target variable (Masa_VAT_cat).

Also the label Vol_VAT will be removed as it is technically the same as Masa_VAT.

In [6]:
#Creating a dataAP dataframe only with Masa_VAT_g and another one with only Masa_VAT_cat

dataAP_g = dataAP.copy()
dataAP_cat = dataAP.copy()

dropcat = ["Masa_VAT_g", "Vol_VAT"]
dropg = ["Masa_VAT_cat", "Vol_VAT"]
dataAP_g = dataAP_g.drop(dropg, axis=1)
dataAP_cat = dataAP_cat.drop(dropcat , axis=1)

#And deleting all the label data from dataALL
dropALL = ["Masa_VAT_cat", "Masa_VAT_g", "Vol_VAT"]
dataALL = dataALL.drop(dropALL, axis=1)

Separating the features X from the label y, next we have the three possible X and y depending on the database we're going to work with

In [7]:
#DATA AP NN
#Separating the target variables (Masa_VAT_g and Vol_VAT) from the independent features
target = ["Masa_VAT_g"]
X = dataAP_g.drop(target, axis=1).values
y = dataAP_g[target].values	

In [8]:
#DATA PHY NN
#Separating the target variables (Masa_VAT_g and Vol_VAT) from the independent features
target = ["Masa_VAT_g"]
X = dataPHY.values
y = dataAP_g[target].values	

In [9]:
#DATA ALL NN
#Separating the target variables (Masa_VAT_g and Vol_VAT) from the independent features
target = ["Masa_VAT_g"]
X = dataALL.values
y = dataAP_g[target].values	

Train-Validation-Test Split

In [10]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now we can use X_train and y_train as your training data
# and X_test and y_test as your test data

In [11]:
print(X_train.shape)
print(y_train.shape)

(118, 33)
(118, 1)


# Creating a model with keras tuner: DataAP

Defining the data used in the training and testing

In [12]:
## Data AP
#Separating the target variables (Masa_VAT_g and Vol_VAT) from the independent features
target = ["Masa_VAT_g"]
X = dataAP_g.drop(target, axis=1).values
y = dataAP_g[target].values	

In [13]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now we can use X_train and y_train as your training data
# and X_test and y_test as your test data

In [14]:
print(X_train.shape)
print(y_train.shape)

(118, 17)
(118, 1)


Installing keras tuner

In [15]:
!pip install keras-tuner --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.4 MB/s eta 0:00:00


In [16]:
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch, Hyperband
import tensorflow as tf
import IPython
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

<ipython-input-16-091a60f0002a>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


In [17]:
class ANNhypermodel(HyperModel):
    
    def __init__(self, input_shape):
        self.input_shape= (17,)
        
    def build(self, hp):
        model= tf.keras.Sequential()
        
        # Tune the number of units in the first Dense layer
        # Defining dense units as a close approx to the original neural network to perform a fair comparision!
        
        
        hp_units_1= hp.Int('units_1', min_value=128, max_value= 160, step=32)
        hp_units_2= hp.Int('units_2', min_value=64, max_value= 128, step=32)

        model.add(tf.keras.layers.Dense(units=hp_units_1, activation='relu', input_shape= self.input_shape))
        model.add(tf.keras.layers.Dense(units=hp_units_2, activation='relu'))
        model.add(tf.keras.layers.Dense(1))

        # Tune the learning rate for the optimizer 
        hp_learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default= 0.0005)

        model.compile(loss='mse',
                    optimizer= tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), metrics= ['mae','mse']
                     )

        return model

hypermodel= ANNhypermodel(input_shape= [len(X_train)])

2. Instantiate the tuner to perform hypertuning

In [18]:
MAX_TRIALS = 20
EXECUTION_PER_TRIAL = 2
tuner= RandomSearch(hypermodel,
               objective= 'val_mse',
               max_trials= MAX_TRIALS,
               executions_per_trial= EXECUTION_PER_TRIAL,
               directory= 'random_search',
               project_name='houseprices',
               overwrite=True)

tuner.search_space_summary()

Search space summary
Default search space size: 3
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 160, 'step': 32, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0005, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [19]:
HYPERBAND_MAX_EPOCHS = 150
EXECUTION_PER_TRIAL = 2

tuner= Hyperband(hypermodel,
                   objective= 'val_mse',
                   max_epochs=HYPERBAND_MAX_EPOCHS, #Set 100+ for good results
                   executions_per_trial=EXECUTION_PER_TRIAL,
                   directory= 'hyperband',
                   project_name='houseprices',
                   overwrite=True)

tuner.search_space_summary()

Search space summary
Default search space size: 3
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 160, 'step': 32, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0005, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


Run the hyperparameter search.

In [20]:
import time

print('searching for the best params!')

t0= time.time()
tuner.search(x= X_train,
             y= y_train,
             epochs=100,
             batch_size= 64,
             validation_data= (X_test, y_test),
             verbose=0,
             callbacks= []
            )
print(time.time()- t0," secs")

# Retreive the optimal hyperparameters
best_hps= tuner.get_best_hyperparameters(num_trials=1)[0]

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

searching for the best params!


1349.1612300872803  secs


In [21]:
print(f"""
The hyperparameter search is complete. The optimal number of units in the 
first densely-connected layer is {best_hps.get('units_1')},
second layer is {best_hps.get('units_2')} 

and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

# Evaluate the best model.
print(best_model.metrics_names)
loss, mae, mse = best_model.evaluate(X_test, y_test)
print(f'loss:{loss} mae: {mae} mse: {mse}')


The hyperparameter search is complete. The optimal number of units in the 
first densely-connected layer is 160,
second layer is 96 

and the optimal learning rate for the optimizer
is 0.0076208407593745765.

[]
1/1 [==============================] - 0s 353ms/step - loss: 0.0202 - mae: 0.1153 - mse: 0.0202
loss:0.020223401486873627 mae: 0.11528563499450684 mse: 0.020223401486873627


Retrain the model with the optimal hyperparameters from the search

In [22]:
# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Build the model with the optimal hyperparameters
tuned_model = tuner.hypermodel.build(best_hps)

# Train the model with early stopping
t00 = time.time()
history_tuned = tuned_model.fit(X_train, y_train, 
                                epochs=200, 
                                validation_data=(X_test, y_test),
                                verbose=0,
                                callbacks=[early_stopping])

print("\n Using Early stopping, needed only", len(history_tuned.history['val_mse']), "epochs to converge!")


 Using Early stopping, needed only 14 epochs to converge!


In [23]:
y_pred_train_tuned= tuned_model.predict(X_train).flatten()
y_pred_test_tuned= tuned_model.predict(X_test).flatten()

print("Training accuracy: ",r2_score(y_train, y_pred_train_tuned))

print("Test accuracy: ",r2_score(y_test, y_pred_test_tuned))

print("Test mean-squared error: ",np.sqrt(mean_squared_error(y_test, y_pred_test_tuned)))

1/1 [==============================] - 0s 27ms/step
Training accuracy:  0.28809270326799796
Test accuracy:  0.24579536576063976
Test mean-squared error:  0.15867082889883857


From the hyperparameter tuning we got that:

The optimal number of units in the densely-connected layers is:
- First layer: 160
- Second layer: 96

and the optimal learning rate for the optimizer
is 0.0076208407593745765.

And using early stopping only 14 epochs were needed to converge.

# Creating a model with keras tuner: DataPHY

Defining the data used in the training and testing

In [24]:
#DATA PHY NN
#Separating the target variables (Masa_VAT_g and Vol_VAT) from the independent features
target = ["Masa_VAT_g"]
X = dataPHY.values
y = dataAP_g[target].values	

In [25]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now we can use X_train and y_train as your training data
# and X_test and y_test as your test data

In [26]:
print(X_train.shape)
print(y_train.shape)

(118, 16)
(118, 1)


In [27]:
!pip install keras-tuner --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch, Hyperband
import tensorflow as tf
import IPython
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [29]:
class ANNhypermodel(HyperModel):
    
    def __init__(self, input_shape):
        self.input_shape= (16,)
        
    def build(self, hp):
        model= tf.keras.Sequential()
        
        # Tune the number of units in the first Dense layer
        # Defining dense units as a close approx to the original neural network to perform a fair comparision!
        
        
        hp_units_1= hp.Int('units_1', min_value=128, max_value= 160, step=32)
        hp_units_2= hp.Int('units_2', min_value=64, max_value= 128, step=32)

        model.add(tf.keras.layers.Dense(units=hp_units_1, activation='relu', input_shape= self.input_shape))
        model.add(tf.keras.layers.Dense(units=hp_units_2, activation='relu'))
        model.add(tf.keras.layers.Dense(1))

        # Tune the learning rate for the optimizer 
        hp_learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default= 0.0005)

        model.compile(loss='mse',
                    optimizer= tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), metrics= ['mae','mse']
                     )

        return model

hypermodel= ANNhypermodel(input_shape= [len(X_train)])

2. Instantiate the tuner to perform hypertuning

In [30]:
MAX_TRIALS = 20
EXECUTION_PER_TRIAL = 2
tuner= RandomSearch(hypermodel,
               objective= 'val_mse',
               max_trials= MAX_TRIALS,
               executions_per_trial= EXECUTION_PER_TRIAL,
               directory= 'random_search',
               project_name='houseprices',
               overwrite=True)

tuner.search_space_summary()

Search space summary
Default search space size: 3
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 160, 'step': 32, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0005, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [31]:
HYPERBAND_MAX_EPOCHS = 150
EXECUTION_PER_TRIAL = 2

tuner= Hyperband(hypermodel,
                   objective= 'val_mse',
                   max_epochs=HYPERBAND_MAX_EPOCHS, #Set 100+ for good results
                   executions_per_trial=EXECUTION_PER_TRIAL,
                   directory= 'hyperband',
                   project_name='houseprices',
                   overwrite=True)

tuner.search_space_summary()

Search space summary
Default search space size: 3
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 160, 'step': 32, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0005, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


Run the hyperparameter search.

In [32]:
import time

print('searching for the best params!')

t0= time.time()
tuner.search(x= X_train,
             y= y_train,
             epochs=100,
             batch_size= 64,
             validation_data= (X_test, y_test),
             verbose=0,
             callbacks= []
            )
print(time.time()- t0," secs")

# Retreive the optimal hyperparameters
best_hps= tuner.get_best_hyperparameters(num_trials=1)[0]

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

searching for the best params!
1650.080234527588  secs


In [33]:
print(f"""
The hyperparameter search is complete. The optimal number of units in the 
first densely-connected layer is {best_hps.get('units_1')},
second layer is {best_hps.get('units_2')} 

and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

# Evaluate the best model.
print(best_model.metrics_names)
loss, mae, mse = best_model.evaluate(X_test, y_test)
print(f'loss:{loss} mae: {mae} mse: {mse}')


The hyperparameter search is complete. The optimal number of units in the 
first densely-connected layer is 160,
second layer is 64 

and the optimal learning rate for the optimizer
is 0.006528791394798441.

[]
1/1 [==============================] - 1s 656ms/step - loss: 0.0129 - mae: 0.0941 - mse: 0.0129
loss:0.012929310090839863 mae: 0.09407304972410202 mse: 0.012929310090839863


Retrain the model with the optimal hyperparameters from the search

In [34]:
# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Build the model with the optimal hyperparameters
tuned_model = tuner.hypermodel.build(best_hps)

# Train the model with early stopping
t00 = time.time()
history_tuned = tuned_model.fit(X_train, y_train, 
                                epochs=200, 
                                validation_data=(X_test, y_test),
                                verbose=0,
                                callbacks=[early_stopping])

print("\n Using Early stopping, needed only", len(history_tuned.history['val_mse']), "epochs to converge!")


 Using Early stopping, needed only 12 epochs to converge!


In [35]:
y_pred_train_tuned= tuned_model.predict(X_train).flatten()
y_pred_test_tuned= tuned_model.predict(X_test).flatten()

print("Training accuracy: ",r2_score(y_train, y_pred_train_tuned))

print("Test accuracy: ",r2_score(y_test, y_pred_test_tuned))

print("Test mean-squared error: ",np.sqrt(mean_squared_error(y_test, y_pred_test_tuned)))

1/1 [==============================] - 0s 32ms/step
Training accuracy:  0.37088301841259097
Test accuracy:  0.4068863020834368
Test mean-squared error:  0.14070886803514687


From the hyperparameter tuning we got that:

The optimal number of units in the densely-connected layers is:
- First layer: 160
- Second layer: 64

and the optimal learning rate for the optimizer
is 0.006528791394798441.

And using early stopping only 12 epochs were needed to converge.

# Creating a model with keras tuner: DataALL

Defining the data used in the training and testing

In [36]:
# #And deleting all the label data from dataALL (just in case)
# dropALL = ["Masa_VAT_cat", "Masa_VAT_g", "Vol_VAT"]
# dataALL = dataALL.drop(dropALL, axis=1)

In [37]:
#DATA ALL NN
#Separating the target variables (Masa_VAT_g and Vol_VAT) from the independent features
target = ["Masa_VAT_g"]
X = dataALL.values
y = dataAP_g[target].values	

In [38]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now we can use X_train and y_train as your training data
# and X_test and y_test as your test data

In [39]:
print(X_train.shape)
print(y_train.shape)

(118, 33)
(118, 1)


In [40]:
!pip install keras-tuner --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch, Hyperband
import tensorflow as tf
import IPython
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [42]:
class ANNhypermodel(HyperModel):
    
    def __init__(self, input_shape):
        self.input_shape= (33,)
        
    def build(self, hp):
        model= tf.keras.Sequential()
        
        # Tune the number of units in the first Dense layer
        # Defining dense units as a close approx to the original neural network to perform a fair comparision!
        
        
        hp_units_1= hp.Int('units_1', min_value=128, max_value= 160, step=32)
        hp_units_2= hp.Int('units_2', min_value=64, max_value= 128, step=32)

        model.add(tf.keras.layers.Dense(units=hp_units_1, activation='relu', input_shape= self.input_shape))
        model.add(tf.keras.layers.Dense(units=hp_units_2, activation='relu'))
        model.add(tf.keras.layers.Dense(1))

        # Tune the learning rate for the optimizer 
        hp_learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default= 0.0005)

        model.compile(loss='mse',
                    optimizer= tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), metrics= ['mae','mse']
                     )

        return model

hypermodel= ANNhypermodel(input_shape= [len(X_train)])

2. Instantiate the tuner to perform hypertuning

In [43]:
MAX_TRIALS = 20
EXECUTION_PER_TRIAL = 2
tuner= RandomSearch(hypermodel,
               objective= 'val_mse',
               max_trials= MAX_TRIALS,
               executions_per_trial= EXECUTION_PER_TRIAL,
               directory= 'random_search',
               project_name='houseprices',
               overwrite=True)

tuner.search_space_summary()

Search space summary
Default search space size: 3
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 160, 'step': 32, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0005, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [44]:
HYPERBAND_MAX_EPOCHS = 150
EXECUTION_PER_TRIAL = 2

tuner= Hyperband(hypermodel,
                   objective= 'val_mse',
                   max_epochs=HYPERBAND_MAX_EPOCHS, #Set 100+ for good results
                   executions_per_trial=EXECUTION_PER_TRIAL,
                   directory= 'hyperband',
                   project_name='houseprices',
                   overwrite=True)

tuner.search_space_summary()

Search space summary
Default search space size: 3
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 160, 'step': 32, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0005, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


Run the hyperparameter search.

In [45]:
import time

print('searching for the best params!')

t0= time.time()
tuner.search(x= X_train,
             y= y_train,
             epochs=100,
             batch_size= 64,
             validation_data= (X_test, y_test),
             verbose=0,
             callbacks= []
            )
print(time.time()- t0," secs")

# Retreive the optimal hyperparameters
best_hps= tuner.get_best_hyperparameters(num_trials=1)[0]

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

searching for the best params!


1893.022055864334  secs


In [46]:
print(f"""
The hyperparameter search is complete. The optimal number of units in the 
first densely-connected layer is {best_hps.get('units_1')},
second layer is {best_hps.get('units_2')} 

and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

# Evaluate the best model.
print(best_model.metrics_names)
loss, mae, mse = best_model.evaluate(X_test, y_test)
print(f'loss:{loss} mae: {mae} mse: {mse}')


The hyperparameter search is complete. The optimal number of units in the 
first densely-connected layer is 128,
second layer is 128 

and the optimal learning rate for the optimizer
is 0.004386943776968041.

[]
1/1 [==============================] - 0s 247ms/step - loss: 0.0110 - mae: 0.0826 - mse: 0.0110
loss:0.010983134619891644 mae: 0.08261366933584213 mse: 0.010983134619891644


Retrain the model with the optimal hyperparameters from the search

In [47]:
# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Build the model with the optimal hyperparameters
tuned_model = tuner.hypermodel.build(best_hps)

# Train the model with early stopping
t00 = time.time()
history_tuned = tuned_model.fit(X_train, y_train, 
                                epochs=200, 
                                validation_data=(X_test, y_test),
                                verbose=0,
                                callbacks=[early_stopping])

print("\n Using Early stopping, needed only", len(history_tuned.history['val_mse']), "epochs to converge!")


 Using Early stopping, needed only 45 epochs to converge!


In [48]:
y_pred_train_tuned= tuned_model.predict(X_train).flatten()
y_pred_test_tuned= tuned_model.predict(X_test).flatten()

print("Training accuracy: ",r2_score(y_train, y_pred_train_tuned))

print("Test accuracy: ",r2_score(y_test, y_pred_test_tuned))

print("Test mean-squared error: ",np.sqrt(mean_squared_error(y_test, y_pred_test_tuned)))

1/1 [==============================] - 0s 62ms/step
Training accuracy:  0.8320683716051002
Test accuracy:  0.5591770164508776
Test mean-squared error:  0.12130664931971893


From the hyperparameter tuning we got that:

The optimal number of units in the densely-connected layers is:
- First layer: 128
- Second layer: 128

and the optimal learning rate for the optimizer
is 0.004386943776968041.

And using early stopping only 45 epochs were needed to converge.